## Import Dependencies

In [17]:
import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import *
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
%matplotlib inline

## Data augmentation

In [3]:
batch_size = 4

# data augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,    
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data',  # this is the target data directory
        target_size=(124, 124),  # all images will be resized to 124 x 124
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 734 images belonging to 2 classes.


In [11]:
x, y = next(train_generator)
x.shape, y.shape

((4, 124, 124, 3), (4,))

## Build Model

In [43]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(124, 124, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Train the model

In [44]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=5)
model.save_weights('weights.h5')  # save weights after training

Epoch 1/5
500/500 [==============================] - 190s 381ms/step - loss: 0.4161 - acc: 0.8125
Epoch 2/5
500/500 [==============================] - 125s 249ms/step - loss: 0.3499 - acc: 0.8150
Epoch 3/5
500/500 [==============================] - 164s 328ms/step - loss: 0.3263 - acc: 0.8190
Epoch 4/5
500/500 [==============================] - 105s 210ms/step - loss: 0.3191 - acc: 0.8410
Epoch 5/5
500/500 [==============================] - 100s 201ms/step - loss: 0.3118 - acc: 0.8280


## Cross check

In [70]:
x, y = next(train_generator)
y = y.reshape(len(y), 1)

y_pred = model.predict(x)
y_pred = (y_pred > 0.5) * 1

y == y_pred

array([[ True],
       [ True],
       [ True],
       [False]], dtype=bool)

In [47]:
img_path = random.choice(glob('data/1/*'))
img = load_img(img_path, target_size=(124, 124, 3)) # this is a PIL image
x = img_to_array(img) / 255.0
y = model.predict(np.expand_dims(x, axis=0))
print(np.squeeze(y) > 0.5)
img